### astuce sirine ###

In [ ]:
df2 = df2.sort(f.col('networkid').desc(), f.col('softwareVersion').desc())
df_fin = df2.dropDuplicates(subset = ['networkid'])

df_agg_nombre = df_agg_nombre.withColumn('band',f.lit("0"))

df_netw_band_agg_tot.write.mode('append').format('parquet').partitionBy("day", "data_type").saveAsTable('req_stc.ecms_metrics_agg_day_network_kpi')
 
df_rssi_agg_tot_count = df_rssi_agg_tot_count.withColumn('percent', (f.round((f.col('nombre_client')/f.col('sum(nombre_client)')),4)))
    
df_household = df_household.withColumn("date" , f.from_unixtime(f.col("time") , "yyyy-MM-dd HH:mm:ss"))


In [ ]:
sauvegarde dans une table 
df_tot.write.mode('append').format('parquet').partitionBy("day", "data_type").saveAsTable('req_stc.ecms_metrics_network_day')    


### récupération automatique des données ###

In [ ]:
from datetime import datetime, timedelta, date 
df_ini = spark.read.parquet("/warehouse/tablespace/external/hive/req_stc.db/ssa_ecms_kpi_nombre_day_eole") ## max de la table de suavegarde finale

max_day = df_ini.agg(f.max('day')).toPandas().iloc[0][0]
day_ref = (date.today()-max_day).days-1

In [ ]:
src_ecms   = spark.read.parquet("/warehouse/tablespace/external/hive/req_stc.db/ecms_household_day_rc2_v2").filter(f.col('day') >= f.date_sub(f.current_date(),day_ref+1))
src_metrics   = spark.read.parquet("/warehouse/tablespace/external/hive/req_stc.db/ecms_metrics_network_day").filter(f.col('day') >= f.date_sub(f.current_date(),day_ref+1))

src_ecms_dmax = src_ecms.agg(f.max('day')).toPandas().iloc[0][0]
src_metrics_dmax = src_metrics.filter(f.col('data_type')==0).agg(f.max('day')).toPandas().iloc[0][0]

min_day = min([src_ecms_dmax
               ,src_metrics_dmax
               ]) 
 
day_j = (date.today()-min_day).days

day_j


In [ ]:
while  day_ref >=  day_j :    
    

        ########### calcul nombre de clients à partir de la base de données customer base ############
    df = src_ecms.filter(src_ecms.day == f.date_sub(f.current_date(),day_ref)
                                    ).select('day','networkid').filter(src_ecms.networkid.like('127%')).distinct()
                                    
    day_ref = day_ref - 1 

### window ###

In [ ]:
windowSpec = Window.partitionBy('day_UTFR', 'networkid', 'childID'
          ).orderBy(f.col('last_sn').asc())


data_2_4 = data_2_4.withColumn("cnx_ID_avant",f.lag("connectionId",1).over(windowSpec))
data_2_4 = data_2_4.withColumn("lag_time",f.lag("last_sn",1).over(windowSpec))

In [ ]:
df_tot = df_tot[['day', 'networkid', 'band', 'data_type', 'data_type_value', 'data_type_interval']]

w = Window.partitionBy('day', 'networkid')

df_tot = df_tot.withColumn('data_type_interval_tot', f.max('data_type_interval').over(w)
                                ).withColumn('data_type_value_max', f.max('data_type_value').over(w))

In [ ]:
grp_window = Window.partitionBy('networkid')
magic_percentile = f.expr('percentile_approx(data_type_interval_int, 0.5)')

df_tot_count_filtred = df_tot_count_filtred.withColumn('med_val', magic_percentile.over(grp_window))

In [ ]:
windowSpec = Window.partitionBy('day_UTFR', 'networkid', 'childID'
          ).orderBy(f.col('last_sn').asc())


data_2_4 = data_2_4.withColumn("cnx_ID_avant",f.lag("connectionId",1).over(windowSpec))
data_2_4 = data_2_4.withColumn("lag_time",f.lag("last_sn",1).over(windowSpec))
data_2_4 = data_2_4.withColumn("trafic_avant",f.lag("max_trafic",1).over(windowSpec))

In [ ]:
wd = Window.partitionBy('day_UTFR', 'networkid', 'childID', 'connectionId', 'band'
      ).orderBy(f.col('time').asc()).rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_donne_tot = df_donne_tot.withColumn ("last_sn", f.last('UTCFRA', True).over(wd))

date automatique

In [ ]:
from datetime import datetime, timedelta

def date_range(start, end):
    delta = end - start  # as timedelta
    days = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(delta.days + 1)]
    return days


start_date_1 = datetime(2022, 2, 3)
end_date_1 = datetime(2022,2, 5)
    
days_1= date_range(start_date_1, end_date_1)

liste_dates = [*days_1]

In [ ]:
df_statique = df_statique.withColumn('day', f.from_unixtime(f.unix_timestamp('date', 'dd/MM/yyyy'))) convert timestamp

In [ ]:
### concaténer plusieurs éléments  ###

df_channel_band_5.groupby("IMEI").agg(f.concat_ws(", ", f.collect_list(df_channel_band_5.day_5)).alias('day_5'),
                                            f.concat_ws(", ", f.first(df_channel_band_5.lag_wifi_5_canal_auto), f.collect_list(df_channel_band_5.wifi_5_canal_auto)).alias('wifi_5_canal_auto'))
                                            
                                            
|           IMEI|                 day|wifi_2_4_canal_auto|channel_enabled_2_4|      networkid|               day_5|wifi_5_canal_auto|channel_enabled_5|
+---------------+--------------------+-------------------+-------------------+---------------+--------------------+-----------------+-----------------+
|127213037308043|2021-09-25 00:00:...|       Yes, No, Yes|           statique|127213037308043|2021-09-25 00:00:...|     Yes, No, Yes|         statique|
|127213109613320|2021-09-15 00:00:...|       Yes, No, Yes|           statique|127213109613320|2021-09-15 00:00:...|     Yes, No, Yes|         statique|


In [ ]:
###converison seconde en heures minutes secondes

from time import *
print(strftime('%H %M %S', gmtime(52890)))


In [ ]:
iad_nbr_rep = iad_nbr_rep.withColumn("nbr_rep", \
                      f.when(iad_nbr_rep["nbr_rep_2"] >2 , 2).otherwise(iad_nbr_rep["nbr_rep_2"]))

In [ ]:
spark.sql("show partitions req_stc.ecms_metrics_network_day").show(1500)  #####partion des données
#import shutil
#spark.read.parquet('/warehouse/tablespace/external/hive/c/ecms_metrics_network_day/day=2021-03-17/data_type=0').count()
#/warehouse/tablespace/external/hive/msf_exploratoire.db/dng_fai_client_parquet
#shutil.rmtree('/warehouse/tablespace/external/hive/c/ecms_metrics_network_day/day=2021-03-17/data_type=0')

In [ ]:
### pour modifier une partition bien précise il faut que les colonnes soit dans le méme ordre ###

df.createOrReplaceTempView("temp_view")

spark.sql("insert overwrite table req_stc_dev.ecms_test_mise_a_jour partition (day= '2022-01-22') select data_type,data_type_name,data_type_interval, data_type_interval_name, firstname    from temp_view")

### a on fait withColumn pour changer la valeur de la colonne et on met ###

df.write.mode('Overwrite').save("/warehouse/tablespace/external/hive/req_stc_dev.db/ecms_test_mise_a_jour/day=2022-01-22")

In [ ]:
###converison seconde en heures minutes secondes

from time import *
print(strftime('%H %M %S', gmtime(540017)))


In [ ]:
### exemple airflow avec plusieurs scripts ###

from airflow import DAG
from datetime import datetime, timedelta
from airflow.contrib.operators.kubernetes_pod_operator import KubernetesPodOperator
from airflow.operators.dummy_operator import DummyOperator
from airflow.utils.dates import days_ago

from airflow.operators.email_operator import EmailOperator
from airflow.models import Variable
from airflow.utils.trigger_rule import TriggerRule



default_args = {
    'owner': 'alghali',
    'depends_on_past': False,  
    'email': ['ALGHALI@bouyguestelecom.fr'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=1)
}

dag = DAG(
    'dpps_cqf_agh_TVoverWIFI', 
    default_args=default_args, 
    start_date=datetime.strptime("2022-01-20", "%Y-%M-%d"),
    schedule_interval="0 09 * * *",
    catchup=False)


start = DummyOperator(
    task_id='start',
    dag=dag)


end = DummyOperator(
    task_id='end',
    dag=dag)


DPPS_CQF_YYI_IoW_NGR1 = KubernetesPodOperator(namespace='airflow-prod',
                          image="bt1svpm0.bpa.bouyguestelecom.fr:8084/release/airflow/notebooks:prod",
                          is_delete_operator_pod=True,
                          arguments=["notebooks_dpps/DPPS_CQF_YYI_IoW_NGR1.ipynb","-u","dpps", "--timeout", "86000"],
                          name="DPPS_CQF_YYI_IoW_NGR1",
                          task_id="DPPS_CQF_YYI_IoW_NGR1",
                          image_pull_policy='Always',
                          get_logs=False,
                          service_account_name="airflow",
                          dag=dag
                          )

dpps_cqf_ssa_ToW_vanc_eole_1 = KubernetesPodOperator(namespace='airflow-prod',
                          image="bt1svpm0.bpa.bouyguestelecom.fr:8084/release/airflow/notebooks:prod",
                          is_delete_operator_pod=True,
                          arguments=["notebooks_dpps/dpps_cqf_ssa_ToW_vanc_eole_1.ipynb","-u","dpps", "--timeout", "86000"],
                          name="dpps_cqf_ssa_ToW_vanc_eole_1",
                          task_id="dpps_cqf_ssa_ToW_vanc_eole_1",
                          image_pull_policy='Always',
                          get_logs=False,
                          service_account_name="airflow",
                          dag=dag
                          )

dpps_cqf_agh_ToW_vanc_eole_2 = KubernetesPodOperator(namespace='airflow-prod',
                          image="bt1svpm0.bpa.bouyguestelecom.fr:8084/release/airflow/notebooks:prod",
                          is_delete_operator_pod=True,
                          arguments=["notebooks_dpps/dpps_cqf_agh_ToW_vanc_eole_2.ipynb","-u","dpps", "--timeout", "86000"],
                          name="dpps_cqf_agh_ToW_vanc_eole_2",
                          task_id="dpps_cqf_agh_ToW_vanc_eole_2",
                          image_pull_policy='Always',
                          get_logs=False,
                          service_account_name="airflow",
                          dag=dag
                          )


AllTaskSuccess = EmailOperator (
    dag=dag,
    trigger_rule=TriggerRule.ALL_SUCCESS,
    task_id="AllTaskSuccess",
    to=["ALGHALI@bouyguestelecom.fr"],
    subject="Dag TVoWIFI completed successfully",
    html_content='<h5>Hello Alaeddine,<br><br>TV over WIFI dag completed </h5><h4><font color="green">successfully.</font></h4><br><br><h5>Regards</h5>')


Task_Failed = EmailOperator (
    dag=dag,
    trigger_rule=TriggerRule.ONE_FAILED,
    task_id="Task_Failed",
    to=["ALGHALI@bouyguestelecom.fr"],
    subject="TVoWIFI Task Failed",
    html_content='<h3>Task Failed</h3>')

start >> [DPPS_CQF_YYI_IoW_NGR1, dpps_cqf_ssa_ToW_vanc_eole_1] 
dpps_cqf_ssa_ToW_vanc_eole_1 >> dpps_cqf_agh_ToW_vanc_eole_2
[DPPS_CQF_YYI_IoW_NGR1, dpps_cqf_ssa_ToW_vanc_eole_1, dpps_cqf_agh_ToW_vanc_eole_2] >> Task_Failed 
[DPPS_CQF_YYI_IoW_NGR1, dpps_cqf_agh_ToW_vanc_eole_2] >> AllTaskSuccess >> end
